In [1]:
import torch
from torch import nn
from torch.optim import Adam
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import os

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
train_df  = pd.read_csv("/kaggle/input/bean-leaf-lesions-classification/train.csv")
val_df = pd.read_csv('/kaggle/input/bean-leaf-lesions-classification/val.csv')
train_df['image:FILE'] = "/kaggle/input/bean-leaf-lesions-classification/" + train_df['image:FILE']
val_df['image:FILE'] = "/kaggle/input/bean-leaf-lesions-classification/" + val_df['image:FILE']

In [4]:
train_df.head()

,image:FILE,category
0,/kaggle/input/bean-leaf-lesions-classification...,0
1,/kaggle/input/bean-leaf-lesions-classification...,0
2,/kaggle/input/bean-leaf-lesions-classification...,0
3,/kaggle/input/bean-leaf-lesions-classification...,0
4,/kaggle/input/bean-leaf-lesions-classification...,0


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1034 entries, 0 to 1033
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   image:FILE  1034 non-null   object
 1   category    1034 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 16.3+ KB


In [6]:
transform  = transforms.Compose([
    transforms.Resize([128,128]),
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float)
])

In [7]:
class custom_dataset(Dataset):
    def __init__(self, dataframe, transform = None):
        self.dataframe = dataframe
        self.transform = transform
        self.labels = torch.tensor(dataframe['category']).to(device)
    def __len__(self):
        return self.dataframe.shape[0]
    def __getitem__(self, idx):
        image_path = self.dataframe.iloc[idx, 0]
        label = self.labels[idx]

        image = Image.open(image_path)

        if transform :
            image = (self.transform(image)/255.0).to(device)

        return image, label

In [8]:
 train_dataset = custom_dataset(dataframe=train_df, transform= transform)
 val_dataset = custom_dataset(dataframe=val_df, transform= transform)

In [9]:
lr = 1e-3
batch_size = 8
epochs=15

In [10]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [11]:
googlenet = models.googlenet(weights='DEFAULT')

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:01<00:00, 46.5MB/s]


In [12]:
for param in googlenet.parameters():
    param.requires_grad = True

In [13]:
googlenet.fc

Linear(in_features=1024, out_features=1000, bias=True)

In [14]:
num_features = len(train_df['category'].unique())

In [15]:
googlenet.fc = nn.Linear(googlenet.fc.in_features, num_features)

In [16]:
googlenet.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [17]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(googlenet.parameters(), lr=lr)

In [22]:
total_loss_train_plot = []
total_acc_train_plot = []

for epoch in range(epochs):
    total_loss_train = 0
    total_acc_train = 0
    for inputs, label in train_dataloader:
        optimizer.zero_grad()
        pred = googlenet(inputs)
        loss = criterion(pred, label)

        loss.backward()
        optimizer.step()
        
        total_loss_train += loss.item()

        total_acc_train += (torch.argmax(pred,axis=1)==label).sum().item()

    total_loss_train_plot.append(round(total_loss_train/1000, 4))
    total_acc_train_plot.append(round((total_acc_train/train_dataset.__len__())*100, 4))

    print(f"epoch:{epoch}, train_loss:{round(total_loss_train/1000, 4)}, train_acc:{round((total_acc_train/train_dataset.__len__())*100, 4)}")
        
        

epoch:0, train_loss:0.1031, train_acc:63.4429
epoch:1, train_loss:0.0849, train_acc:72.6306
epoch:2, train_loss:0.0739, train_acc:77.5629
epoch:3, train_loss:0.0697, train_acc:79.4004
epoch:4, train_loss:0.0665, train_acc:79.1103
epoch:5, train_loss:0.0607, train_acc:81.4313
epoch:6, train_loss:0.0637, train_acc:82.3985
epoch:7, train_loss:0.0642, train_acc:81.528
epoch:8, train_loss:0.0507, train_acc:86.2669
epoch:9, train_loss:0.0435, train_acc:87.911
epoch:10, train_loss:0.0518, train_acc:85.2998
epoch:11, train_loss:0.0407, train_acc:88.7814
epoch:12, train_loss:0.0418, train_acc:86.9439
epoch:13, train_loss:0.0341, train_acc:90.4255
epoch:14, train_loss:0.0363, train_acc:89.8453


In [24]:
with torch.no_grad():
    total_loss_test = 0
    total_acc_test = 0
    for inputs, labels in val_dataloader:
        output = googlenet(inputs)
        loss = criterion(output, labels)
        total_loss_test += loss.item()
        total_acc_test += (torch.argmax(output,axis=1)==labels).sum().item()

print(round((total_acc_test/val_dataset.__len__())*100, 4))

86.4662


# transfer_learning


In [28]:
googlenet = models.googlenet(weights='DEFAULT')
for param in googlenet.parameters():
    param.requires_grad = False

googlenet.fc = nn.Linear(googlenet.fc.in_features, num_features)

googlenet.fc.requires_grad =  True
googlenet.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [29]:
criterion = nn.CrossEntropyLoss()
optimizer = Adam(googlenet.parameters(), lr=lr)

In [30]:
total_loss_train_plot = []
total_acc_train_plot = []

for epoch in range(epochs):
    total_loss_train = 0
    total_acc_train = 0
    for inputs, label in train_dataloader:
        optimizer.zero_grad()
        pred = googlenet(inputs)
        loss = criterion(pred, label)

        loss.backward()
        optimizer.step()
        
        total_loss_train += loss.item()

        total_acc_train += (torch.argmax(pred,axis=1)==label).sum().item()

    total_loss_train_plot.append(round(total_loss_train/1000, 4))
    total_acc_train_plot.append(round((total_acc_train/train_dataset.__len__())*100, 4))

    print(f"epoch:{epoch}, train_loss:{round(total_loss_train/1000, 4)}, train_acc:{round((total_acc_train/train_dataset.__len__())*100, 4)}")
        
        

epoch:0, train_loss:0.1286, train_acc:51.7408
epoch:1, train_loss:0.1121, train_acc:60.2515
epoch:2, train_loss:0.1086, train_acc:63.0561
epoch:3, train_loss:0.0979, train_acc:67.118
epoch:4, train_loss:0.0992, train_acc:66.1509
epoch:5, train_loss:0.0987, train_acc:65.4739
epoch:6, train_loss:0.0948, train_acc:67.3114
epoch:7, train_loss:0.0939, train_acc:67.795
epoch:8, train_loss:0.0983, train_acc:68.3752
epoch:9, train_loss:0.092, train_acc:69.8259
epoch:10, train_loss:0.0911, train_acc:68.6654
epoch:11, train_loss:0.093, train_acc:67.6983
epoch:12, train_loss:0.0922, train_acc:67.2147
epoch:13, train_loss:0.0932, train_acc:70.3095
epoch:14, train_loss:0.09, train_acc:70.8897


In [31]:
with torch.no_grad():
    total_loss_test = 0
    total_acc_test = 0
    for inputs, labels in val_dataloader:
        output = googlenet(inputs)
        loss = criterion(output, labels)
        total_loss_test += loss.item()
        total_acc_test += (torch.argmax(output,axis=1)==labels).sum().item()

print(round((total_acc_test/val_dataset.__len__())*100, 4))

66.1654
